<a href="https://colab.research.google.com/github/itberrios/CV_tracking/blob/main/setup_tutorials/tutorial_yolo_nas_and_ocsort.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **YOLO-NAS + OC-SORT**

This notebook contains a tutorial that shows how to incorporate YOLO-NAS with OC-SORT to perform real time visual tracking on a data stream. In this case, we will use a YouTube video to simulate a data stream of image frames.

## **YOLO-NAS**
[YOLO-NAS](https://github.com/Deci-AI/super-gradients/blob/master/YOLONAS.md) is a powerful object detector with an optimal neural network architecture that has been selected using Neural Architecture Search (NAS), hence the name NAS. 

At the time of release, it outperforms all of the other single shot object detectos in terms of speed and accuracy. It also excels in an area that most single shot detector struggle with, small objects. Two-stage detectors typically perform better than single stage detectors on small objects, at the cost of increased detection time [source](https://arxiv.org/pdf/1907.09408.pdf). YOLO-NAS, howver seems to provide a good tradeoff between detection speed and accuracy on small objects that previous versions of YOLO have not been able to deliver.

## **OC-SORT**
[OC-SORT](https://arxiv.org/abs/2203.14360) is a robust visual obejct tracking algorithm that improves upon the already popular [SORT](https://arxiv.org/abs/1602.00763) algorithm.

SORT tends to loose track on obejcts when they are lost for extended periods of time or when non-linear motion occurs. Algorithms such as Deep SORT have effectoively improved SORT in these scenarios with a Deep Association metric that is computed with a [Siamese Neural Network](https://arxiv.org/pdf/1707.02131.pdf) over the image patches. Eventhough this is effective it comes with the cost of increased detection time due to the deep association and the Siamese network needs to be trained on in-domain data for this approach to be effective. OC-SORT on the otherhand is able to effectively increase tracking performance in a model free fashion with minimal impact to inference speed.

OC-SORT introduces
- Observation Centric Re-Update (ORU)
    - Reduces accumulated Kalman Filter error/uncertainty when a lost track is re-associated
- Observation Centric Momentum (OCM)
    - Uses previous observations to compute a low noise expected motion direction and incorporates it into the track association cost
- Observation Centric Recovery (OCR)
    - Uses the last known observation as a secondary association to help prevent lost tracks

For more details and a break down of each technique that OC-SORT introduces, please see this [article](https://medium.com/@itberrios6/introduction-to-ocsort-c1ea1c6adfa2).

#### Install Libraries

In [ ]:
# sketchy fix for "https://stackoverflow.com/questions/73711994/importerror-cannot-import-name-is-directory-from-pil-util-usr-local-lib"
!pip install fastcore -U

In [ ]:
# supergradients installs
! pip install torch==1.11.0+cu113 torchvision==0.12.0+cu113 torchaudio==0.11.0 --extra-index-url https://download.pytorch.org/whl/cu113 &> /dev/null
! pip install super-gradients
! pip install pytorch-quantization==2.1.2 --extra-index-url https://pypi.ngc.nvidia.com &> /dev/null
! pip install matplotlib==3.1.3 &> /dev/null
! pip install --upgrade psutil==5.9.2 &> /dev/null
! pip install --upgrade pillow==7.1.2 &> /dev/null

In [ ]:
!pip install filterpy
!pip install pytube
!pip install moviepy
!pip install ffmpeg

# bug fix for imageio-ffmpeg
!pip install imageio==2.4.1

### Get OCSORT code

In [ ]:
!mkdir ocsort
%cd ocsort
!wget https://raw.githubusercontent.com/noahcao/OC_SORT/master/trackers/ocsort_tracker/ocsort.py 
!wget https://raw.githubusercontent.com/noahcao/OC_SORT/master/trackers/ocsort_tracker/kalmanfilter.py 
!wget https://raw.githubusercontent.com/noahcao/OC_SORT/master/trackers/ocsort_tracker/association.py 

%cd ..

### Import Libraries

In [291]:
import os
import time
from tqdm import tqdm

import numpy as np
import cv2
import filterpy

import torch
import super_gradients as sg
import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams["figure.figsize"] = (20, 10)

## Download video from YouTube

In [7]:
from pytube import YouTube

# current bug in PYTube prevents this code from working

url = r"https://www.youtube.com/watch?v=JteKbauGolo"
yt = YouTube(url)
print("Video Title: ", yt.title)

# download video
video_path = yt.streams \
  .filter(progressive=True, file_extension='mp4') \
  .order_by('resolution') \
  .desc() \
  .first() \
  .download() 

Video Title:  Chastain BANZAI last lap move!!


## Instantiate YOLO-NAS model

In [134]:
from super_gradients.training import models
from super_gradients.common.object_names import Models

model = models.get("yolo_nas_s", pretrained_weights="coco").cuda()

### Quantize model for better speed

FOR SOME REASON THIS ACTUALLY REDUCES THE SPEED BY A FACTOR OF 10!

In [9]:
from super_gradients.training.utils.quantization.selective_quantization_utils import SelectiveQuantizer


# cls_model = models.get(model_name="resnet50", pretrained_weights="imagenet")

q_util = SelectiveQuantizer(
    default_quant_modules_calibrator_weights="max",
    default_quant_modules_calibrator_inputs="histogram",
    default_per_channel_quant_weights=True,
    default_learn_amax=False,
    verbose=True,
)
q_util.quantize_module(model)

## Perform inference on Simulated Video Stream

#### First we will use MoviePy to get the frame rate and save the audio for later

In [292]:
from moviepy.editor import VideoFileClip

videoclip = VideoFileClip(video_path)
audioclip = videoclip.audio

video_fps = videoclip.fps
video_fps

29.97002997002997

## Instantiate tracker object

In [275]:
from ocsort import ocsort

tracker = ocsort.OCSort(det_thresh=0.25)

Helper function for bounding box colors

In [277]:
import colorsys    

def get_color(number):
    """ Converts an integer number to a color """
    # change these however you want to
    hue = number*30 % 180
    saturation = number*103 % 256
    value = number*50 % 256

    # expects normalized values
    color = colorsys.hsv_to_rgb(hue/179, saturation/255, value/255)

    return [int(c*255) for c in color]

## Now we can simualate the data stream using opencv

Make sure to reset the tracker each time you run the inference

In [315]:
# get frame info for tracker and video saving 
h, w = (720, 1280)
h2, w2 = h//2, w//2
# h2, w2 = 640, 640
img_size = (h2, w2) 
img_info = (h2, w2)

In [ ]:
# ensure tracker is reset
tracker = ocsort.OCSort(det_thresh=0.35, max_age=20, delta_t=5, inertia=0.1, use_byte=True)

cap = cv2.VideoCapture(video_path)

if (cap.isOpened() == False):
    print("Error opening video file")

frames = []
i = 0
counter, fps, elapsed = 0, 0, 0
start_time = time.perf_counter()

while(cap.isOpened()):

  # read each video frame
  ret, frame = cap.read()

  if ret == True:

    # read image and resize by half
    og_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = cv2.resize(og_frame, 
                       (w2, h2), interpolation=cv2.INTER_LINEAR)

    # perform inference on small frame
    predictions = model.predict(images=[frame], iou=0.25, conf=0.35)

    # get detections
    img_preds = list(predictions._images_prediction_lst)[0]
    dets = np.hstack((img_preds.prediction.bboxes_xyxy, 
                      np.c_[img_preds.prediction.confidence]))

    # update tracker
    tracks = tracker.update(dets, img_info, img_size)

    # draw tracks on frame
    for track in tracker.trackers:
      
      track_id = track.id
      hits = track.hits
      color = get_color(track_id*15)
      x1,y1,x2,y2 = np.round(track.get_state()).astype(int).squeeze()*2

      cv2.rectangle(og_frame, (x1,y1),(x2,y2), color, 2)
      cv2.putText(og_frame, 
                  f"{track_id}-{hits}", 
                  (x1+10,y1+10), 
                  cv2.FONT_HERSHEY_SIMPLEX, 
                  0.5,
                  color, 
                  1,
                  cv2.LINE_AA)
      
    # update FPS and place on frame
    # elapsed = (time.perf_counter() - start_time)
    # fps = np.round((i+1)/elapsed)
    current_time = time.perf_counter()
    elapsed = (current_time - start_time)
    counter += 1
    if elapsed > 1:
        fps = counter / elapsed;
        counter = 0;
        start_time = current_time;


    cv2.putText(og_frame, 
                f"FPS: {np.round(fps, 2)}", 
                (10,h - 10), 
                cv2.FONT_HERSHEY_SIMPLEX, 
                1,
                (255,255,255), 
                2,
                cv2.LINE_AA)

    # append to list
    frames.append(og_frame)

    # # TEMP for debug
    # if i == 10:
    #   break
    # else:
    #   i += 1

  # Break the loop
  else:
    break

# When everything done, release
# the video capture object
cap.release()
 
# Closes all the frames
cv2.destroyAllWindows()

In [ ]:
plt.imshow(frames[-1])

## Now let's put this in a video

In [312]:
# save to mp4
out = cv2.VideoWriter('chastain_wall_ride.mp4',
                      cv2.VideoWriter_fourcc(*'MP4V'), 
                      video_fps,
                      (w, h))
 
for frame in frames:
    out.write(cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))
    
out.release()
del out

### Now add back the audio

In [314]:
from moviepy.editor import CompositeAudioClip
detection_video = VideoFileClip('chastain_wall_ride.mp4')

# add sound and save
detection_video.audio = CompositeAudioClip([audioclip])
detection_video.write_videofile('chastain_wall_ride_with_audio.mp4')

Moviepy - Building video chastain_wall_ride_with_audio.mp4.
MoviePy - Writing audio in chastain_wall_ride_with_audioTEMP_MPY_wvf_snd.mp3


t:   0%|          | 4/1049 [00:00<00:26, 39.63it/s, now=None]

MoviePy - Done.
Moviepy - Writing video chastain_wall_ride_with_audio.mp4

